In [1]:
from tkinter import *
from tkinter import messagebox
from PIL import ImageTk, Image
import glob
from enum import Enum
import os.path
import threading
import time
import numpy as np
import re
from google.cloud import vision
from google.cloud.vision import types
import cv2
import pandas as pd

In [2]:
class Category(Enum):
    NAME = 0
    KILL = 1
    DS = 2

In [3]:
class Dialog():
    def __init__(self, parent, category):
        self.parent = parent
        self.frame = Frame(self.parent)
        self.category = category
        self.old_x, old_y = (0, 0)
        self.new_x, new_y = (0, 0)
        self.drag_area = None

        self.img = Image.open(images[0])
        w, h = self.img.size
        self.img = self.img.resize((int(w/2), int(h/2)))
        self.image = ImageTk.PhotoImage(self.img)

        w, h = self.img.size
        self.canvas = Canvas(parent, width=w, height=h)
        self.canvas.grid(column=0, row=0)
        self.canvas.create_image(0, 0, anchor=NW, image=self.image)
        self.canvas.image = self.image

        self.parent.bind("<Button-1>", self.motion1)
        self.parent.bind("<ButtonRelease-1>", self.motion2)
        self.parent.bind("<B1-Motion>", self.motion3)

    def motion1(self, event):
        self.canvas.delete(self.drag_area)
        self.drag_area = None
        self.old_x, self.old_y = event.x, event.y

    def motion2(self, event):
        self.new_x, self.new_y = event.x, event.y
        msg = messagebox.askokcancel("확인", "제대로 영역지정을 하셨나요?")

        if msg:
            self.parent.destroy()

            if self.category == Category.NAME:
                application.set_nameCanvas()
            elif self.category == Category.KILL:
                application.set_killCanvas()
            elif self.category == Category.DS:
                application.set_dsCanvas()


    def motion3(self, event):
        if self.drag_area != None:
            self.canvas.delete(self.drag_area)

        w = 2
        self.drag_area = self.canvas.create_rectangle(self.old_x, self.old_y, event.x, event.y, width=w)

In [4]:
class Main(Frame):
    def __init__(self, parent):
        self.parent = parent
        self.frame = Frame(self.parent)

        btn_edit_img = Button(parent, text='이미지 편집', command=self.crop_image)
        btn_edit_img.grid(column=0, row=0, padx=5, pady=5)
        self.lbl_status1 = Label(parent, text='진행 상황')
        self.lbl_status1.grid(column=1, row=0, padx=5, pady=5)
        
        btn_detect_img = Button(parent, text="이미지 인식", command=self.detect_image)
        btn_detect_img.grid(column=2, row=0, padx=5, pady=5)
        self.lbl_status2 = Label(parent, text='진행 상황')
        self.lbl_status2.grid(column=3, row=0, padx=5, pady=5)
        
        btn_export = Button(parent, text="엑셀파일로", command=self.export_excel)
        btn_export.grid(column=4, row=0, padx=5, pady=5)
        
        self.i = 0
        self.job_crop = None
        self.job_detect = None
        
        self.name = list()
        self.power = list()
        self.kill = list()
        self.dead = list()
        self.support = list()
        self.t1 = list()
        self.t2 = list()
        self.t3 = list()
        self.t4 = list()
        self.t5 = list()
        
        self.df = None

        # Name
        btn_get_name = Button(parent, text='이름 & 투력 & 총 킬수', command=self.open_nameDialog)
        btn_get_name.grid(column=0, row=1, padx=5, pady=5)

        # Kill
        btn_get_kill = Button(parent, text='티어별 킬수', command=self.open_killDialog)
        btn_get_kill.grid(column=0, row=2, padx=5, pady=5)

        # Dead & Support
        btn_get_ds = Button(parent, text='전사 & 원조', command=self.open_dsDialog)
        btn_get_ds.grid(column=0, row=3, padx=5, pady=5)

    def open_nameDialog(self):
        self.nameDialog = Toplevel(self.parent)
        self.name_app = Dialog(self.nameDialog, Category.NAME)
    def set_nameCanvas(self):
        img = Image.open(images[0]).crop((self.name_app.old_x*2, self.name_app.old_y*2, self.name_app.new_x*2, self.name_app.new_y*2))

        w, h = img.size
        self.canvas_name = Canvas(self.parent, width=w/2, height=h/2)
        self.canvas_name.grid(column=1, row=1, padx=1, pady=5, columnspan=4)

        img = img.resize((int(w/2), int(h/2)))
        image = ImageTk.PhotoImage(img)
        self.canvas_name.create_image(0, 0, anchor=NW, image=image)
        self.canvas_name.image = image

    def open_killDialog(self):
        self.killDialog = Toplevel(self.parent)
        self.kill_app = Dialog(self.killDialog, Category.KILL)
    def set_killCanvas(self):
        img = Image.open(images[0]).crop(
            (self.kill_app.old_x * 2, self.kill_app.old_y * 2, self.kill_app.new_x * 2, self.kill_app.new_y * 2))

        w, h = img.size
        self.canvas_kill = Canvas(self.parent, width=w / 2, height=h / 2)
        self.canvas_kill.grid(column=1, row=2, padx=1, pady=5, columnspan=4)

        img = img.resize((int(w / 2), int(h / 2)))
        image = ImageTk.PhotoImage(img)
        self.canvas_kill.create_image(0, 0, anchor=NW, image=image)
        self.canvas_kill.image = image

    def open_dsDialog(self):
        self.dsDialog = Toplevel(self.parent)
        self.ds_app = Dialog(self.dsDialog, Category.DS)
    def set_dsCanvas(self):
        img = Image.open(images[0]).crop(
            (self.ds_app.old_x * 2, self.ds_app.old_y * 2, self.ds_app.new_x * 2, self.ds_app.new_y * 2))

        w, h = img.size
        self.canvas_ds = Canvas(self.parent, width=w / 2, height=h / 2)
        self.canvas_ds.grid(column=1, row=3, padx=1, pady=5, columnspan=4)

        img = img.resize((int(w / 2), int(h / 2)))
        image = ImageTk.PhotoImage(img)
        self.canvas_ds.create_image(0, 0, anchor=NW, image=image)
        self.canvas_ds.image = image

    def crop_image(self):
        total = len(images)

        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status1.config(text=s)
        self.parent.update()

        initial = Image.open(images[self.i])
        width = initial.size[0]
        height = initial.size[1]

        path, file_name = os.path.split(images[self.i])

        # name
        crop_name = initial.crop((self.name_app.old_x * 2, self.name_app.old_y * 2, self.name_app.new_x * 2, self.name_app.new_y * 2))
        img = np.array(crop_name) 
        img = img[:, :, ::-1].copy() 
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
        gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

        img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

        ret, img_thresh = cv2.threshold(
            img_blurred, 
            190, #name : 180
            255,
            cv2.THRESH_BINARY
        )
        cv2.imwrite('data/info/croped/name/' + file_name, img_thresh)

        # kill
        crop_kill = initial.crop((self.kill_app.old_x * 2, self.kill_app.old_y * 2, self.kill_app.new_x * 2, self.kill_app.new_y * 2))
        
        lower_blue = (0, 0, 0)
        upper_blue =(255, 255, 60)

        img = np.array(crop_kill) 
        img = img[:, :, ::-1].copy() 

        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img_mask = cv2.inRange(img_hsv, lower_blue, upper_blue)
        img_result = cv2.bitwise_not(img, img)
        img_result1 = cv2.bitwise_and(img_result, img_result, mask=img_mask)
        cv2.imwrite('data/info/croped/kill/' + file_name, img_result1)


        # Dead & Support
        crop_ds = initial.crop((self.ds_app.old_x * 2, self.ds_app.old_y * 2, self.ds_app.new_x * 2, self.ds_app.new_y * 2))
        crop_ds.save('data/info/croped/ds/' + file_name)
        
        self.i += 1
            
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status1.config(text=s)
        self.parent.update()
        
        if self.i == len(images):
            self.parent.after_cancel(self.job_crop) 
            self.i = 0
            messagebox.showinfo("작업 상태", "완료!")
        else:
            self.job_crop = self.parent.after(0, self.crop_image)
            
    def detect_image(self):
        total = len(images)
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status2.config(text=s)
        self.parent.update()
        
        path, file_name = os.path.split(images[self.i])
    
        # name
        detect_name = self.detect_text('data/info/croped/name/' + file_name)
        detect_name = self.change_to_list(detect_name)
        self.name.append(detect_name[0])
        self.power.append(self.change_to_number(detect_name[-2]))
        self.kill.append(self.change_to_number(detect_name[-1]))

        # kill
        detect_kill = self.detect_text('data/info/croped/kill/' + file_name)
        detect_kill = self.change_to_list(detect_kill)

        self.t1.append(self.change_to_number(detect_kill[0]))
        self.t2.append(self.change_to_number(detect_kill[1]))
        self.t3.append(self.change_to_number(detect_kill[2]))
        self.t4.append(self.change_to_number(detect_kill[3]))
        self.t5.append(self.change_to_number(detect_kill[4]))

        # ds
        detect_ds = self.detect_text('data/info/croped/ds/' + file_name)
        detect_ds = self.change_to_list(detect_ds)
        self.dead.append(self.change_to_number(detect_ds[detect_ds.index('전사')+1]))
        self.support.append(self.change_to_number(detect_ds[detect_ds.index('원조')+1]))
        
        self.i += 1
        
        s = '{}/{} 완료'.format(self.i, total)
        self.lbl_status2.config(text=s)
        self.parent.update()
        
        if self.i == len(images):
            self.parent.after_cancel(self.job_detect) 
            self.i = 0
            messagebox.showinfo("작업 상태", "완료!")
        else:
            self.job_detect = self.parent.after(0, self.detect_image)
            
    def export_excel(self):
        self.df = pd.DataFrame({'name': self.name,
                                   'power':self.power,
                                   'all_kill':self.kill,
                                   'T1_kill':self.t1,
                                   'T2_kill':self.t2,
                                   'T3_kill':self.t3,
                                   'T4_kill':self.t4,
                                   'T5_kill':self.t5,
                                   'dead':self.dead,
                                   'support':self.support})
        
        self.df.to_excel('data/info/fin1.xlsx')
            
    def detect_text(self, path):
        """Detects text in the file."""
        from google.cloud import vision
        import io
        client = vision.ImageAnnotatorClient()

        # [START vision_python_migration_text_detection]
        with io.open(path, 'rb') as image_file:
            content = image_file.read()

        image = vision.types.Image(content=content)

        response = client.text_detection(image=image)
        texts = response.text_annotations

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                        for vertex in texts[0].bounding_poly.vertices])

        return texts[0].description

    def change_to_number(self, s_power):
        list_of_numbers = re.findall(r'\d+', s_power)
        return int(''.join(list_of_numbers))
    
    def change_to_list(self, chars):
        tmp_texts = list()
        s = ""
        for c in chars:
            if c != '\n':
                s += c
            else:
                tmp_texts.append(s)
                s = ""

        return tmp_texts
        

In [5]:
if __name__ == "__main__":
    root = Tk()
    application = Main(root)
    root.resizable(width=False, height=False)
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROKS-78e082836307.json"
    images = glob.glob('data/info/original/*.jpg')

    root.mainloop()